# ML-8. Metrics learning


В данной задаче мы попробуем кластеризовать покупателей продуктовой компании на основе его персональных данных, таких как состав семьи, доход, вовлеченность в маркетинговые компании, интересы и предпочтения по разным категориям товаров.

In [ ]:
#! pip install yellowbrick

: 

In [ ]:
#Загрузим библиотеки
import numpy as np
import pandas as pd
import datetime
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt, numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import AgglomerativeClustering
from matplotlib.colors import ListedColormap
from sklearn import metrics
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
np.random.seed(42)

 ### Описание данных
 



* **ID**: Идентификационный номер покупателя
* **Year_Birth**: Год рождения покупателя
* **Education**: Степень образования покупателя 
* **Marital_Status**: Семейный  статус покупателя
* **Income**: Годовой доход покупателя
* **Kidhome**: Количество детей до 13 лет, проживающих с покупателем
* **Teenhome**: Количество детей с 13 до 18, проживающих с покупателем
* **Dt_Customer**: Дата регистрации покупателя как клиента компании 
* **Recency**: Количество дней с последней покупки клиента
* **Complain**: Бинарный признак наличия жалоб от клиента в течение двух лет

* **MntWines**: Количество денег, которые клиент потратил на вино за последние два года 
* **MntFruits**: Количество денег, которые клиент потратил на фрукты за последние два года
* **MntMeatProducts**: Количество денег, которые клиент потратил на мясо за последние два года
* **MntFishProducts**: Количество денег, которые клиент потратил на рыбу за последние два года
* **MntSweetProducts**: Количество денег, которые клиент потратил на сладости за последние два года
* **MntGoldProds**: Количество денег, которые клиент потратил на золотые украшения за последние два года

* **NumDealsPurchases**: Количество покупок товаров со скидкой
* **AcceptedCmp1**: Бинарный признак того, что клиент был привлечен в 1 маркетинговую кампанию
* **AcceptedCmp2**: Бинарный признак того, что клиент был привлечен в 2 маркетинговую кампанию
* **AcceptedCmp3**: Бинарный признак того, что клиент был привлечен в 3 маркетинговую кампанию
* **AcceptedCmp4**: Бинарный признак того, что клиент был привлечен в 4 маркетинговую кампанию
* **AcceptedCmp5**: Бинарный признак того, что клиент был привлечен в 5 маркетинговую кампанию
* **Response**: Бинарный признак того, что клиент откликнулся на последнюю прошедшую маркетинговую кампанию

* **NumWebPurchases**: Количество покупок через сайт компании 
* **NumCatalogPurchases**: Количество покупок через каталог 
* **NumStorePurchases**: Количество покупок через магазин
* **NumWebVisitsMonth**: Количество посещений сайта компании за последний месяц 

### Загрузка и базовая предобработка данных

In [ ]:
#Загрузим и посмотрим на исходные данные 
data = pd.read_csv("marketing_campaign.csv",sep="\t")
data.head()

In [ ]:
#Расссмотрим столбцы поподробнее
data.info()

In [ ]:
data.isna().sum()

**Некоторые первые выводы:**

* Есть пропуски в доходе (`Income`)
* Dt_Customer - признак, который показывает когда клиент попал в базу данных фирмы не является временной меткой, а имеет тип `object`
* В данных присуствуют категориальные признаки, которые надо будет закодировать. 

Для простоты, удалим все данные с пропусками.

In [ ]:
data = data.dropna()
print(f"Количество объектов после удаления пропусков: {data.shape[0]}")

Посмотрим, что именно содержится в столбце "Dt_Customer"

In [ ]:
data['Dt_Customer'].head()

Переведем **"Dt_Customer"** в численный, посмотрим на самую раннюю и самую позднюю запись. 

In [ ]:
data["Dt_Customer"] = pd.to_datetime(data["Dt_Customer"]) 
dt_max = data["Dt_Customer"].max()
dt_min = data["Dt_Customer"].min()
print(f"Дата добавления последнего покупателя в базу: {dt_max}")
print(f"Дата добавления первого покупателя в базу: {dt_min}")


### Feature Engineering

Добавим признак, показывающий, сколько дней покупатель является клиентом фирмы. Для этого вычтем из даты фиксации последнего клиента дату фиксации каждого из клиентов.

In [9]:
data["Customer_For"] = dt_max - data["Dt_Customer"] 
data["Customer_For"] = pd.to_numeric(data["Customer_For"], errors="coerce")

**Следующим шагом, добавим еще пару признаков:**

* Сделаем возраст покупателя на момент сбора данных **"Age"** на основе **"Year_Birth"**, учтем, что данные датируются 14 годом, что и учтем при генерации нашего признака.
* Сделаем признак **"Spent"** показывающий, сколько потратил покупатель на различные категории за последние два года.
* Сделаем признак **"Living_With"** из **"Marital_Status"**, показывающий живет ли с кем-то покупатель.
* Сделаем признак **"Children"** показывающий суммарное количество детей в семье.
* Сделаем признак количество членов семьи **"Family_Size"**.
* Сделаем бинарный признак **"Is_Parent"**, показывающий, что покупатель является родителем.
* Уменьшим количество значений признаков в **"Education"** до 3-х.

In [ ]:
# Воспользуемся нашим списком, чтобы в дальнейшем значения заменить 
data["Marital_Status"].unique()

In [ ]:
# Воспользуемся нашим списком, чтобы в дальнейшем значения заменить
data["Education"].unique()

In [13]:
data["Age"] = 2014-data["Year_Birth"]

data["Spent"] = data["MntWines"]+ data["MntFruits"]+ data["MntMeatProducts"]+ data["MntFishProducts"]+ data["MntSweetProducts"]+ data["MntGoldProds"]

#Здесь есть значения, которые надо перевести в значения Alone или Parthner для бинаризации признака
data["Living_With"]=data["Marital_Status"].replace({"Married":"Partner", "Together":"Partner", "Absurd":"Alone", "Widow":"Alone", "YOLO":"Alone", "Divorced":"Alone", "Single":"Alone",})

data["Children"]=data["Kidhome"]+data["Teenhome"]

data["Family_Size"] = data["Living_With"].replace({"Alone": 1, "Partner":2})+ data["Children"]

data["Is_Parent"] = np.where(data.Children> 0, 1, 0)

data["Education"]=data["Education"].replace({"Basic":"Undergraduate","2n Cycle":"Undergraduate", "Graduation":"Graduate", "Master":"Postgraduate", "PhD":"Postgraduate"})

data=data.rename(columns={"MntWines": "Wines","MntFruits":"Fruits","MntMeatProducts":"Meat","MntFishProducts":"Fish","MntSweetProducts":"Sweets","MntGoldProds":"Gold"})

#Удалим использованные признаки
to_drop = ["Marital_Status", "Dt_Customer", "Z_CostContact", "Z_Revenue", "Year_Birth", "ID"]
data = data.drop(to_drop, axis=1)

In [ ]:
data.describe()

### Работа с выбросами

Посмотрим на распределения признаков на основе информации, является ли покупатель родителем или нет.

In [ ]:
to_plot = ["Income", "Recency", "Customer_For", "Age", "Spent", "Is_Parent"]
plt.figure()
sns.pairplot(data[to_plot], hue= "Is_Parent");
plt.show();

Удалим некоторые выбросы по возрасту `Age` и по заработку `Income`. Удалим покупателей старше 90 лет и зарабатывающий более 600000$.

In [ ]:
data = data[(data["Age"]<90)]
data = data[(data["Income"]<600000)]
print(f"Количество объектов после удаления выбросов: {data.shape[0]}")

### Коррелялицоный анализ

Посмотрим на корреляцию между численными признаками

In [ ]:
plt.figure(figsize=(20,20))  
sns.heatmap(data.corr(),fmt='.2f',annot=True, center=0);

### Кодирование признаков

Закодируем категориальные признаки. Так как у нас их мало (`Education` и`Living_With`), можем использовать смело `LabelEncoder()`

In [18]:
LE=LabelEncoder()
data['Education'] = LE.fit_transform(data['Education'])
data['Living_With'] = LE.fit_transform(data['Living_With'])

In [19]:
#сделаем копию данных
ds = data.copy()
# удалим бинарные признаки
cols_del = ['AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1','AcceptedCmp2', 'Complain', 'Response']
ds = ds.drop(cols_del, axis=1)
#Scaling
scaler = StandardScaler()
scaler.fit(ds)
scaled_ds = pd.DataFrame(scaler.transform(ds),columns= ds.columns )

In [ ]:
#Посмотрим на датасет, который будем использовать для дальнейшей кластеризации
scaled_ds.head()

### Снижение размерности



Будем использовать PCA, снизим размерность до 3, что бы можно было визуализировать результаты кластеризации. Подобно `StandardScaler()` можем вызвать методы `fit()`, `transform()` и их суперпозицию `fit_tranform()`

In [ ]:
pca = PCA(n_components=3)
PCA_ds = pd.DataFrame(pca.fit_transform(scaled_ds), columns=(["col1","col2", "col3"]))
PCA_ds.describe().T

Разобьем колонки по осям для визуализации:

In [22]:
x,y,z = PCA_ds["col1"],PCA_ds["col2"],PCA_ds["col3"]

 Теперь мы можем визуализировать полученные ранее данные, что бы понять, сколько кластеров мы имеем в данных

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection="3d")
ax.scatter(x,y,z, c="blue", marker="o" )
ax.set_title("Визуализация данных после снижения размерности")
ax.set_xlabel('1 проекция PCA')
ax.set_ylabel('2 проекция PCA')
ax.set_zlabel('3 проекция  PCA')
plt.show()

In [24]:
pal = ["y","g", "b","r"]

cmap = colors.ListedColormap(["y","g", "b","r"])

### Построение модели кластеризации

Как мы знаем, методу `k-means` на вход необходимо подать количество кластеров. Но дело в том, что нередко, например как в данной задаче, истинное количество кластеров попросту неизвестно. Для этого нам поможет `Elbow Method`. `Distortion score` есть квадрат расстояния от точки до ее центра кластера. Оптимальным на графике ниже количество кластеров считается, если после него `distortion score` падает практически линейно.

In [ ]:
Elbow_M = KElbowVisualizer(KMeans(), k=10)
Elbow_M.fit(PCA_ds)
Elbow_M.show();

Как видно из графика, 4 оптимальное количество кластеров для нашей задачи. Обучим модель на 4-х кластерах.

In [26]:
model = KMeans(n_clusters=4)
y_pred = model.fit_predict(PCA_ds)
PCA_ds["Clusters"] = y_pred
data["Clusters"]= y_pred

Посмотрим на визуализацию кластеров:

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = plt.subplot(111, projection='3d')
ax.scatter(x, y, z, s=40, c=PCA_ds["Clusters"],cmap=cmap, marker='o')
ax.set_title("Визуализация кластеров")
ax.set_xlabel('1 проекция PCA')
ax.set_ylabel('2 проекция PCA')
ax.set_zlabel('3 проекция  PCA')
plt.show()

### Анализ результатов кластеризации


Самая важная часть после кластеризации - интерпретировать схожесть объектов в каждом из кластеров, оценить распределения.

In [ ]:
pl = sns.countplot(x=data["Clusters"], palette=pal)
pl.set_title("Distribution Of The Clusters")
plt.show()

Разбиения кластеров приблизительно равны, это хорошо, посмотрим на траты и заработок:

In [ ]:
pl = sns.scatterplot(data = data,x=data["Spent"], y=data["Income"],hue=data["Clusters"],palette=pal)
pl.set_title("Cluster's Profile Based On Income And Spending")
plt.legend()
plt.show()

**Доходы и расходы по группам:**

* Группа 0: группа с низкими расходами и средним доходом;

* Группа 1: группа с высокими расходами по расходам и наибольшим доходом;

* Группа 2: группа с низкими расходами и низким доходом;

* Группа 3: группа со средними расходами и средним доходом.

Для подтверждения посмотрим на ящики с усами от трат:


In [ ]:
plt.figure()
pl=sns.swarmplot(x=data["Clusters"], y=data["Spent"], color= "#CBEDDD", alpha=0.5 )
pl=sns.boxenplot(x=data["Clusters"], y=data["Spent"], palette=pal)
plt.show()


Таким образом, мы можем выделить, что больше всех тратит группа в которой больше всего заработок.


### Самостоятельная работа
В качестве самостоятельного задания слушателям предлагается уточнить портрет из каждого кластера, посмотрев на распределения по:
1. количеству детей
2. размеру семьи
3. возрасту 

In [ ]:
pl = sns.barplot(data = data,x=data["Children"], y=data["Spent"],hue=data["Clusters"],palette=pal)
pl.set_title("Cluster's Profile Based On Income And Spending")
plt.legend()
plt.show()

In [ ]:
plt.figure()
pl=sns.swarmplot(x=data["Children"], y=data["Spent"], color= "#CBEDDD", alpha=0.5 )
pl=sns.boxenplot(x=data["Children"], y=data["Spent"], palette=pal)
plt.show()

In [ ]:
pl = sns.barplot(data = data,x=data["Family_Size"], y=data["Spent"],hue=data["Clusters"],palette=pal)
pl.set_title("Cluster's Profile Based On Income And Spending")
plt.legend()
plt.show()

In [ ]:
plt.figure()
pl=sns.swarmplot(x=data["Family_Size"], y=data["Spent"], color= "#CBEDDD", alpha=0.5 )
pl=sns.boxenplot(x=data["Family_Size"], y=data["Spent"], palette=pal)
plt.show()

In [ ]:
pl = sns.scatterplot(data = data,x=data["Spent"], y=data["Age"],hue=data["Clusters"],palette=pal)
pl.set_title("Cluster's Profile Based On Income And Spending")
plt.legend()
plt.show()

In [ ]:
plt.figure()
pl=sns.swarmplot(x=data["Clusters"], y=data["Age"], color= "#CBEDDD", alpha=0.5 )
pl=sns.boxenplot(x=data["Clusters"], y=data["Age"], palette=pal)
plt.show()